In [ ]:
using Plots
using LinearAlgebra
using Krylov
using Printf
using LaTeXStrings
using BenchmarkTools
include("poisson2d.jl")

In [ ]:
default(lw=2, markersize=6,
    xtickfont=font(12), ytickfont=font(12),
    guidefont=font(14), legendfont=font(12), titlefont=font(12))

# Poisson Equation Example
Solve
$$
-\Delta u = f
$$
over $(0,L_x)\times (0,L_y)$ with zero Dirichlet boundary conditions.

Test with the eigenfunctions
$$
u = \sin(k_x \pi/L_x x)\sin(k_y \pi y/L_y)
$$
where $k_x,k_y\in \mathbb{Z}$, since these satisfy the boudnary conditions and
$$
-\Delta u = [(k_x \pi/L_x)^2 + (k_y\pi/L_y)^2] u
$$

In [ ]:
Lx = 1;
Ly = 1;
nx = 9; # (nx-1)×(ny-1) interior points
ny = 9;

x = LinRange(0, Lx, nx + 2)
y = LinRange(0, Ly, ny + 2)

@show Δx = x[2] - x[1];
@show Δy = y[2] - y[1];


xy = [[x_, y_] for y_ in y, x_ in x]; # mesh including boundary points
xy_int = [[x_, y_] for y_ in y[2:end-1], x_ in x[2:end-1]]; # interior mesh points

In [ ]:
A = assemble_laplacian2d(Δx, Δy, nx, ny);

In [ ]:
A

In [ ]:
kx = 3;
ky = 2;
# X = (x,y) in 2D
u_exact = X -> sin(kx * π * X[1] / Lx) * sin(ky * π * X[2] / Ly);
f = X -> ((kx * π / Lx)^2 + (ky * π / Ly)^2) * u_exact(X);

In [ ]:
B = f.(xy_int)[:]; # flattern into a column vector

In [ ]:
contourf(x, y, u_exact.(xy))# note use of transpose;
xlabel!(L"$x$")
ylabel!(L"$y$")
title!(L"Exact Solution, $u$")

In [ ]:
contourf(x, y, f.(xy))# note use of transpose;
xlabel!(L"$x$")
ylabel!(L"$y$")
title!(L"Data, $f$")

In [ ]:
U = A \ B;
u = reshape(U, nx, ny); # get as a 2D array

In [ ]:
contourf(x[2:end-1], y[2:end-1], u)

In [ ]:
err = norm(U .- u_exact.(xy_int)[:], Inf);
println("Error = ", err)

# Timing and Error

In [ ]:
function poisson_solve(n, kx=1, ky=1)
    Lx = 1
    Ly = 1
    nx = n 
    ny = n
    x = LinRange(0, Lx, nx + 2)
    y = LinRange(0, Ly, ny + 2)

    Δx = x[2] - x[1]
    Δy = y[2] - y[1]

    kx = 1
    ky = 1
    u_exact = X -> sin(kx * π * X[1] / Lx) * sin(ky * π * X[2] / Ly)
    f = X -> ((kx * π / Lx)^2 + (ky * π / Ly)^2) * sin(kx * π * X[1] / Lx) * sin(ky * π * X[2] / Ly)
    xy_int = [[x_, y_] for y_ in y[2:end-1],x_ in x[2:end-1]] # interior mesh points
    B = f.(xy_int)[:] # flattern into a column vector
    A = assemble_laplacian2d(Δx, Δy, nx, ny)
    U = A \ B
    err = norm(U .- u_exact.(xy_int)[:], Inf)
    return err
end

In [ ]:
n_vals = [10, 20, 40, 80, 160, 320];
for n in n_vals
    @btime poisson_solve($n);
end

In [ ]:
function poisson_solve_krylov(n, kx=1, ky=1)
    Lx = 1
    Ly = 1
    nx = n
    ny = n
    x = LinRange(0, Lx, nx + 2)
    y = LinRange(0, Ly, ny + 2)

    Δx = x[2] - x[1]
    Δy = y[2] - y[1]

    kx = 1
    ky = 1
    u_exact = X -> sin(kx * π * X[1] / Lx) * sin(ky * π * X[2] / Ly)
    f = X -> ((kx * π / Lx)^2 + (ky * π / Ly)^2) * sin(kx * π * X[1] / Lx) * sin(ky * π * X[2] / Ly)
    xy_int = [[x_, y_] for y_ in y[2:end-1], x_ in x[2:end-1]] # interior mesh points
    B = f.(xy_int)[:] # flattern into a column vector
    A = assemble_laplacian2d(Δx, Δy, nx, ny)
    U, _= cg(A, B)
    err = norm(U .- u_exact.(xy_int)[:], Inf)
    return err
end

In [ ]:
n_vals = [10, 20, 40, 80, 160, 320];
for n in n_vals
    @btime poisson_solve_krylov($n)
end

In [ ]:
err_vals = Float64[];
for n in n_vals
    push!(err_vals, poisson_solve(n))
end

In [ ]:
scatter(1 ./ n_vals, err_vals, 
    xscale=:log10, yscale=:log10, label="Error")
plot!(1 ./ n_vals, 1 ./ n_vals .^ 2, 
    label=L"$\propto \Delta x^2$")
xlabel!(L"$\Delta x$")